### Ustawienie hardware

In [2]:
import torch

OPTIMIZE_WITH_HARDWARE = True

device = torch.device('cpu')
if OPTIMIZE_WITH_HARDWARE:
    if torch.backends.mps.is_available():
        device = torch.device('mps')
        print(f'Selected device: MPS (Metal Performance Shaders)')
    elif torch.cuda.is_available():
        device = torch.device('cuda')
        print(f'Selected device: GPU with CUDA support')
        print(f'CUDA device name: {torch.cuda.get_device_name()}')
        print(f'CUDA device count: {torch.cuda.device_count()}')
        print(f'CUDA device index: {torch.cuda.current_device()}')
else:
    print(f'Selected device: CPU')
    

Selected device: GPU with CUDA support
CUDA device name: Quadro M1200
CUDA device count: 1
CUDA device index: 0
